# Neural Network test

In [48]:
import nn
import numpy as np
from numpy.random import uniform, randint
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numdifftools as nd

In [123]:
np.random.seed(0)

data = pd.DataFrame()

X_LOW = -5
X_HIGH = 5
N_X = 100

N_samples = 100
N_funcs = 3

data['x'] = range(N_samples)
data['x'] = data['x'].apply(lambda x: np.linspace(X_LOW, X_HIGH, N_X))


def random_func(x):
    i = randint(0, N_funcs)
    expected = [int(j == i) for j in range(N_funcs)]
    expected = np.array(expected)
    if i == 0:
        return uniform() * np.sin(uniform() * x + uniform()) + uniform(), expected
    elif i == 1:
        return uniform() * x + uniform(), expected
    elif i == 2:
        return uniform() * x ** 2 + uniform() * x + uniform(), expected

data[['food', 'expected']] = data['x'].apply(random_func).apply(pd.Series)

plt.show()

mean = np.mean(data['food'])
data['food'] = data['food'].apply(lambda x: x - mean)
data['food'] /= data['food'].apply(np.linalg.norm)

mean = np.mean(data['expected'])
data['expected'] = data['expected'].apply(lambda x: x - mean)
data['expected'] /= data['expected'].apply(np.linalg.norm)

data.head()

,x,food,expected
0,"[-5.0, -4.898989898989899, -4.797979797979798,...","[-0.11090017622427759, -0.10622373343795408, -...","[0.8163230613038502, -0.39358433312864205, -0...."
1,"[-5.0, -4.898989898989899, -4.797979797979798,...","[-0.2357551384342597, -0.22682450127655643, -0...","[-0.48870791946240766, 0.8108108663808127, -0...."
2,"[-5.0, -4.898989898989899, -4.797979797979798,...","[-0.19069284283202756, -0.18208468114161522, -...","[0.8163230613038502, -0.39358433312864205, -0...."
3,"[-5.0, -4.898989898989899, -4.797979797979798,...","[-0.11697546753365462, -0.11105712192693855, -...","[0.8163230613038502, -0.39358433312864205, -0...."
4,"[-5.0, -4.898989898989899, -4.797979797979798,...","[-0.11415006043635081, -0.11023001193718829, -...","[0.8163230613038502, -0.39358433312864205, -0...."


In [124]:
TEST_SIZE = 0.2
data_train = data.sample(frac=(1-TEST_SIZE), random_state=0)
data_test = data.drop(data_train.index)

In [126]:
%load_ext autoreload
%autoreload 2

N_HIDDEN = 10
ETA = 1.e-3
ETA_LIMIT = 1.e-8
N_EPOCHS = 100
ADAPTIVE_ETA = False
ETA_FACTOR = 1.5
RANDOM_SEED = 5
MSG_FREQ = 10
BATCH_SIZE = 0.1

ACT_FUNCS = [nn.sigmoid, nn.sigmoid]

network_shape = (N_X, N_HIDDEN, N_funcs)

neural_network = nn.NeuralNetwork(shape=network_shape, act_funcs=ACT_FUNCS,
                                  eta=ETA, eta_limit=ETA_LIMIT, adaptive_eta=ADAPTIVE_ETA, eta_factor=ETA_FACTOR,
                                  random_seed=RANDOM_SEED)

df_stats = neural_network.train(df=data_train[['food', 'expected']], n_epochs=N_EPOCHS, msg_freq=MSG_FREQ, batch_size=BATCH_SIZE)
df_stats.to_csv('stats.csv', mode='a', header=None)
pd.DataFrame(neural_network.parameters).to_csv('parameters.csv')
df_stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Starting training.

[2023-04-07 13:52:58.695465]: Currently at epoch 75/100
Average duration per epoch: 0.13 s.
Estimated time of finishing: 2023-04-07 13:53:02.045565

Training finished.


,start,stop,E_mean,eta,acc_train,acc_test,est_finish
0,2023-04-07 13:52:48.645146,2023-04-07 13:52:48.773191,0.502471,0.001,0.3,None,None
1,2023-04-07 13:52:48.782193,2023-04-07 13:52:48.906215,0.490751,0.001,0.3,None,None
2,2023-04-07 13:52:48.916205,2023-04-07 13:52:49.043251,0.479878,0.001,0.3,None,None
3,2023-04-07 13:52:49.052235,2023-04-07 13:52:49.178281,0.469805,0.001,0.3,None,None
4,2023-04-07 13:52:49.188269,2023-04-07 13:52:49.350304,0.460481,0.001,0.3,None,None
...,...,...,...,...,...,...,...
95,2023-04-07 13:53:01.786171,2023-04-07 13:53:01.914201,0.33472,0.001,0.425,None,2023-04-07 13:53:02.045565
96,2023-04-07 13:53:01.925199,2023-04-07 13:53:02.054234,0.334641,0.001,0.425,None,2023-04-07 13:53:02.045565
97,2023-04-07 13:53:02.064221,2023-04-07 13:53:02.196263,0.334565,0.001,0.425,None,2023-04-07 13:53:02.045565
98,2023-04-07 13:53:02.205254,2023-04-07 13:53:02.328299,0.334491,0.001,0.425,None,2023-04-07 13:53:02.045565


In [121]:
fig = go.Figure([
    go.Scatter(
        x=df_stats['start'],
        y=df_stats['E_mean'],
        name='E_mean'
    ),
    go.Scatter(
        x=df_stats['start'],
        y=df_stats['acc_train'],
        name='acc_train'
    )
])

fig.show()

In [117]:
px.line(df_stats, x='start', y='acc_train')

In [122]:
neural_network.accuracy(df=data_test[['food', 'expected']])

0.2

In [ ]:
EXPECTED = data_train['expected'].iloc[0]

own = neural_network.gradient(EXPECTED)
other = nd.Gradient(lambda x: neural_network.E(EXPECTED, x))(neural_network.parameters)

px.scatter(zip(own, other))

# Linear regression

In [89]:
test = list(range(10))
test[-3:]

[7, 8, 9]